In [1]:
from pymongo import MongoClient

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']

print('Connected to MongoDB')

Connected to MongoDB


In [7]:
# Contoh Insert Data
employee_data = {
    'name': 'Alice',
    'department': 'Finance',
    'age': 29,
    'salary': 4500
}
collection.insert_one(employee_data)
print('Data inserted')

# Tugas: Insert beberapa data tambahan, lakukan update, serta delete data
collection.update_one(
    {'name': 'Alice'},
    {'$set': {
        'experience': 5,
        'status': 'Full-Time',
        'bonus': 800
    }}
)
print("Data updated!")

collection.delete_one({'name': 'Alice'})
print("Data deleted!")

Data inserted
Data updated!
Data deleted!


In [8]:
# Contoh Query Data
for employee in collection.find():
    print(employee)

{'_id': ObjectId('6902c2a5c8508ba659425dff'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
{'_id': ObjectId('690324338ec4c2a98379d1b5'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
{'_id': ObjectId('69032504e8dfe578038bd441'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
{'_id': ObjectId('690b41f6fec69703762f41c6'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
{'_id': ObjectId('690b41fffec69703762f41c7'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}


In [4]:
# Query Aggregation untuk mencari rata-rata gaji semua karyawan
pipeline = [
    {"$group": {"_id": None, "total_salary": {"$sum": "$salary"}}}
]
for result in collection.aggregate(pipeline):
    print(result)

# Query Aggregation untuk mencari rata-rata gaji per departemen
pipeline = [
    {'$group': {'_id': '$department', 'average_salary': {'$avg': '$salary'}}}
]
for result in collection.aggregate(pipeline):
    print(result)

# Query Aggregation untuk mencari jumlah karyawan per departemen
pipeline = [
    {"$group": {"_id": "$department", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]
for result in collection.aggregate(pipeline):
    print(result)

# Query Aggeragation untuk mencari total dan rata-rata berdasarkan pekerjaan
pipeline = [
    {"$group": {
        "_id": "$status",
        "total_salary": {"$sum": "$salary"},
        "avg_salary": {"$avg": "$salary"}
    }}
]
for result in collection.aggregate(pipeline):
    print(result)


{'_id': None, 'total_salary': 18000}
{'_id': 'Finance', 'average_salary': 4500.0}
{'_id': 'Finance', 'count': 4}
{'_id': None, 'total_salary': 18000, 'avg_salary': 4500.0}


In [4]:
from datetime import datetime
import random

product_collection = db["product"]

# Membuat 20 data produk acak
products = []
categories = ["Electronics", "Clothing", "Food", "Books", "Home"]
for i in range(1, 21):
    product = {
        "product_id": i,
        "name": f"Product_{i}",
        "category": random.choice(categories),
        "price": round(random.uniform(10, 500), 2),
        "stock": random.randint(10, 100),
        "created_at": datetime.now()
    }
    products.append(product)

# Insert ke koleksi MongoDB
product_collection.insert_many(products)
print("20 product data inserted successfully!")

# Menampilkan data keseluruhan
for p in product_collection.find():
    print(p)

20 product data inserted successfully!
{'_id': ObjectId('6902c39cc8508ba659425e00'), 'product_id': 1, 'name': 'Product_1', 'category': 'Books', 'price': 359.51, 'stock': 75, 'created_at': datetime.datetime(2025, 10, 30, 8, 47, 8, 627000)}
{'_id': ObjectId('6902c39cc8508ba659425e01'), 'product_id': 2, 'name': 'Product_2', 'category': 'Clothing', 'price': 383.52, 'stock': 40, 'created_at': datetime.datetime(2025, 10, 30, 8, 47, 8, 627000)}
{'_id': ObjectId('6902c39cc8508ba659425e02'), 'product_id': 3, 'name': 'Product_3', 'category': 'Books', 'price': 145.29, 'stock': 98, 'created_at': datetime.datetime(2025, 10, 30, 8, 47, 8, 627000)}
{'_id': ObjectId('6902c39cc8508ba659425e03'), 'product_id': 4, 'name': 'Product_4', 'category': 'Home', 'price': 213.21, 'stock': 87, 'created_at': datetime.datetime(2025, 10, 30, 8, 47, 8, 627000)}
{'_id': ObjectId('6902c39cc8508ba659425e04'), 'product_id': 5, 'name': 'Product_5', 'category': 'Clothing', 'price': 238.87, 'stock': 44, 'created_at': datetim

In [13]:
# Query Aggregation untuk mencari product di atas harga rata-rata
pipeline = [
    {'$group': {'_id': '$category', 'avg_price': {'$avg': '$price'}}},
    {'$sort': {'avg_price': -1}}
]
for result in product_collection.aggregate(pipeline):
    print(result)

{'_id': 'Electronics', 'avg_price': 290.5921875}
{'_id': 'Food', 'avg_price': 284.31425}
{'_id': 'Books', 'avg_price': 248.7802941176471}
{'_id': 'Home', 'avg_price': 242.83486486486484}
{'_id': 'Clothing', 'avg_price': 225.67945945945945}


In [15]:
# Query Aggregation untuk mencari total stok per kategori
pipeline = [
    {'$group': {'_id': '$category', 'total_stock': {'$sum': '$stock'}}},
]
for result in product_collection.aggregate(pipeline):
    print(result)

{'_id': 'Food', 'total_stock': 2144}
{'_id': 'Electronics', 'total_stock': 1887}
{'_id': 'Clothing', 'total_stock': 2018}
{'_id': 'Books', 'total_stock': 1724}
{'_id': 'Home', 'total_stock': 2194}


In [18]:
# Query Aggregation untuk mencari produk dengan harga tertinggi
pipeline = [
    {'$sort': {'price': -1}},
    {'$limit': 5}
]
for result in product_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('6902c492c8508ba659425e50'), 'product_id': 1, 'name': 'Product_1', 'category': 'Books', 'price': 497.06, 'stock': 62, 'created_at': datetime.datetime(2025, 10, 30, 8, 51, 14, 49000)}
{'_id': ObjectId('6902c433c8508ba659425e1e'), 'product_id': 11, 'name': 'Product_11', 'category': 'Clothing', 'price': 490.1, 'stock': 18, 'created_at': datetime.datetime(2025, 10, 30, 8, 49, 39, 104000)}
{'_id': ObjectId('6902c39cc8508ba659425e0c'), 'product_id': 13, 'name': 'Product_13', 'category': 'Books', 'price': 489.58, 'stock': 23, 'created_at': datetime.datetime(2025, 10, 30, 8, 47, 8, 627000)}
{'_id': ObjectId('69032595e8dfe578038bd474'), 'product_id': 11, 'name': 'Product_11', 'category': 'Books', 'price': 486.84, 'stock': 93, 'created_at': datetime.datetime(2025, 10, 30, 15, 45, 9, 477000)}
{'_id': ObjectId('6902c443c8508ba659425e2a'), 'product_id': 3, 'name': 'Product_3', 'category': 'Electronics', 'price': 485.59, 'stock': 25, 'created_at': datetime.datetime(2025, 10, 30, 8, 